### Лекция 10 - Работа с базами данных

- Выполнение SQL-запросов
- ORM на примере SQLAlchemy
- Немного о noSql на примере Mongo

In [1]:
import sqlite3
# import MySQLdb    - для MySQL
# import psycopg2   - для PostgreSQL
# import pymssql    - для MS SQL

# дальнейший cursor-based код одинаков для всех движков

In [2]:
DBNAME = r'db/catalog.db'


def create_database():
    # создаем базу данных sqlite 
    # (присоединяемся к ней первый раз)
    db = sqlite3.connect(DBNAME)

    with db:
        # Получаем объект курсора
        cursor = db.cursor()
        
        # SQL-запрос #1:
        cursor.execute('''
            CREATE TABLE performers(id INTEGER PRIMARY KEY, 
                                    name TEXT, 
                                    desc TEXT)
        ''')
        db.commit()

        # SQL-запрос #2:
        cursor.execute('''
               CREATE TABLE albums(
               id INTEGER PRIMARY KEY, 
               name TEXT,
               release_year INTEGER,
               perfid INTEGER,
               FOREIGN KEY(perfid) REFERENCES performers(id))
               ''')
        db.commit()


try:
    create_database()
    print('Database was created succesfully!')
except sqlite3.OperationalError as e:
    print(e)

Database was created succesfully!


![img](images/database.png)

In [3]:
def insert_records():
    # присоединяемся к базе данных sqlite
    db = sqlite3.connect(DBNAME)

    with db:
        cursor = db.cursor()

        ans = input('Do you wish to insert some performers? (y/n)')
        while ans == 'y':
            perf = input('Performer: ')
            # Потенциально опасно!
            # НЕ ДЕЛАТЬ ТАК!
            cursor.execute('INSERT INTO performers VALUES(NULL, "%s", "")' % perf)
            ans = input('Continue? (y/n)')
    
        db.commit()
        
        ans = input('Do you wish to insert some albums? (y/n)')
        while ans== 'y':
            perfID = input('Performer ID: ')
            albumname = input('Album name: ')
            albumyear = int(input('Album year: '))
            # Здесь лучше: делаем параметризированный запрос
            cursor.execute('INSERT INTO albums VALUES(NULL, ?, ?, ?)',
                           (albumname, albumyear, perfID))
            ans = input('Continue? (y/n)')

        db.commit()


insert_records()

Do you wish to insert some performers? (y/n)y
Performer: 10cc
Continue? (y/n)y
Performer: Supertramp
Continue? (y/n)y
Performer: Renaissance
Continue? (y/n)n
Do you wish to insert some albums? (y/n)y
Performer ID: 1
Album name: Deceptive Bends
Album year: 1977
Continue? (y/n)y
Performer ID: 2
Album name: Breakfast in America
Album year: 1979
Continue? (y/n)y
Performer ID: 3
Album name: Novella
Album year: 1977
Continue? (y/n)n


In [4]:
def select_records():
    db = sqlite3.connect(DBNAME)

    with db:
        cursor = db.cursor()
        cursor.execute('SELECT id, name from performers')
        performers = cursor.fetchall()
    
        cursor.execute('''
                SELECT performers.name, albums.name, albums.release_year
                from albums INNER JOIN performers ON albums.perfID = performers.id
                ''')
        albums = cursor.fetchall()

        # fetchone():   считывает одну следующую строку результирующего набора запроса.
        #               Результирующий набор - это объект, возвращаемый 
        #               объектом-курсором при выполнении запроса к таблице;
        # fetchall():   считывает все строки в результирующем наборе запроса. 
        #               Если какие-то строки уже были извлечены из результирующего набора, 
        #               то метод возвращает оставшиеся строки набора;
        # rowcount:     Это read-only атрибут, в котором хранится количество строк, 
        #               затронутых при вызове метода execute().

        return performers, albums
    

performers, albums = select_records()

print()
for performer in performers:
    print(performer)
    
print()
for album in albums:
    print(album)


(1, '10cc')
(2, 'Supertramp')
(3, 'Renaissance')

('10cc', 'Deceptive Bends', 1977)
('Supertramp', 'Breakfast in America', 1979)
('Renaissance', 'Novella', 1977)


### ORM (Object-Relational Mapping)

In [5]:
import sqlalchemy
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, ForeignKey
from sqlalchemy.orm import relationship


Base = declarative_base()


class Performer(Base):
    __tablename__ = 'performers'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    desc = Column(String)
    albums = relationship('Album', back_populates='performer')
    
    def __repr__(self):
        return "<Performer(name='%s', desc='%s')>" % (
                        self.name, self.desc)


class Album(Base):
    __tablename__ = 'albums'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    release_year = Column(Integer)
    perfid = Column(Integer, ForeignKey('performers.id'))
    #performer = relationship('Performer', backref='albums')
    performer = relationship('Performer', back_populates='albums')
    
    def __repr__(self):
        return "<Album(name='%s', year='%d')>" % (
                        self.name, self.release_year)


In [6]:
def insert_records_via_ORM():
    ans = input('Do you wish to insert some performers? (y/n)')
    while ans== 'y':
        name = input('Performer: ')
        performer = Performer(name=name, desc='')
        session.add(performer)
        session.commit()

        ans = input('Continue? (y/n)')

    ans = input('Do you wish to insert some albums? (y/n)')
    while ans == 'y':
        performer_name = input('Performer: ')
        performer = session.query(Performer) \
                           .filter(Performer.name == performer_name) \
                           .first()
        if performer:
            album_name = input('Album name: ')
            album_year = int(input('Album year: '))
            album = Album(name=album_name, release_year=album_year)
            performer.albums.append(album)
            session.add(album)
            session.flush()
            session.commit()
        else:    
            print('No such performer!')
            
        ans = input('Continue? (y/n)')


In [7]:
def select_records_via_ORM():
    performers = [performer.name 
                  for performer in session.query(Performer)]
    albums = [(album.performer.name, album.name, album.release_year) 
              for album in session.query(Album)]
    return performers, albums


In [8]:
# Флаг echo включает ведение лога через стандартный модуль logging Питона.
# Когда он включен, мы увидим все созданные нами SQL-запросы.
engine = sqlalchemy.create_engine('sqlite:///' + DBNAME, echo=False)
#engine = sqlalchemy.create_engine('sqlite:///:memory:', echo=True)

Session = sessionmaker(bind=engine)
session = Session()

insert_records_via_ORM()

performers, albums = select_records_via_ORM()

print('\nPerformers:')
for performer in performers:
    print(performer)

print('\nAlbums:')
for album in albums:
    print(album)
    
session.close()

Do you wish to insert some performers? (y/n)y
Performer: Camel
Continue? (y/n)n
Do you wish to insert some albums? (y/n)y
Performer: Camel
Album name: A Nod And A Wink
Album year: 2002
Continue? (y/n)y
Performer: 10cc
Album name: Look Hear?
Album year: 1979
Continue? (y/n)y
Performer: Camel
Album name: Rain Dances
Album year: 1977
Continue? (y/n)n

Performers:
10cc
Supertramp
Renaissance
Camel

Albums:
('10cc', 'Deceptive Bends', 1977)
('Supertramp', 'Breakfast in America', 1979)
('Renaissance', 'Novella', 1977)
('Camel', 'A Nod And A Wink', 2002)
('10cc', 'Look Hear?', 1979)
('Camel', 'Rain Dances', 1977)


In [9]:
albums = [(album.performer.name, album.name, album.release_year) 
              for album in session.query(Album).filter(Album.release_year == 1977)]
albums

[('10cc', 'Deceptive Bends', 1977),
 ('Renaissance', 'Novella', 1977),
 ('Camel', 'Rain Dances', 1977)]

### noSql и mongoDB